In [1]:
import csv
import logging
import os
import math
import glob
import random
import numpy as np
import pandas as pd
import multiprocessing as mp
import data.usc.features as cls_features
from log import LogTimer

In [9]:
num_partitions = 15
partitions = [i for i in range(num_partitions)]
data_dir = '/beegfs/dr2915/sonyc_ust'
annotation_path = os.path.join(data_dir, 'annotations.csv')
annotation_data = pd.read_csv(annotation_path).sort_values('audio_filename')
df = annotation_data[['split', 'audio_filename']].drop_duplicates()

sum_rows = 0
for partition_to_run in partitions:
    if partition_to_run is None:
        row_iter = df
    else:
        n_rows = df.shape[0]
        partition_size = math.ceil(n_rows / num_partitions)
        start_idx = partition_to_run * partition_size
        end_idx = min(n_rows, (partition_to_run + 1) * partition_size)
        df_split = df.iloc[start_idx:end_idx]
        row_iter = df_split
    sum_rows = sum_rows + row_iter.shape[0]
    
print(df.shape[0])
print(sum_rows)

3068
3068


In [6]:
output_dir = '/scratch/sk7898/embeddings/features/sonyc_ust/quantized_l3/default_int8/l3_audio_20200304152812_8000_64_160_1024_half'

In [45]:
df['output_path'] = df['audio_filename'].apply(lambda x: os.path.join(output_dir, os.path.splitext(x)[0] + '.npz'))
output_paths = list(df['output_path'])
list_files = [os.path.join(output_dir, filename) for filename in os.listdir(output_dir)]
rem_files = [x for x in output_paths if x not in list_files]
df_new = df[df['output_path'].isin(rem_files)]

left_rows = 0
for partition_to_run in partitions:
    if partition_to_run is None:
        df_subset = df_new
    else:
        n_rows = df_new.shape[0]
        partition_size = math.ceil(n_rows / num_partitions)
        start_idx = partition_to_run * partition_size
        end_idx = min(n_rows, (partition_to_run + 1) * partition_size)
        df_subset = df_new.iloc[start_idx:end_idx]
    left_rows = left_rows + df_subset.shape[0]
    
print(len(rem_files))
print(left_rows)

2122
2122
